# One-vs-Rest Classifier

This notebook implements an one-vs-rest classifier that fine-tunes several BERT models to tell if a sentence contains problematic metaphors.

<div hidden>
TODO: add extend data3/data.json with better data in the same format that actually makes sense.
</div>

## Imports and Setup

In [ ]:
!pip install transformers -Uqq
!pip install sklearn -Uqq
!pip install datasets -Uqq
!pip install torch -Uqq
!pip install numpy -Uqq
!pip install evaluate -Uqq

In [1]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
MODEL_NAME = "one-vs-rest-bert"

## Loading Dataset

In [3]:
dataset = load_dataset("json", data_files="data3/data.json", field="data")
dataset

Using custom data configuration default-cfdee65a4c7aa8fc
Found cached dataset json (/home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'agency', 'humanComparison', 'hyperbole', 'historyComparison', 'unjustClaims', 'deepSounding', 'sceptics', 'deEmphasize', 'performanceNumber', 'inscrutable'],
        num_rows: 329
    })
})

In [4]:
dataset["train"][0:3]

{'text': ['A new vision of artificial intelligence for the people',
  'The gig workers fighting back against the algorithms',
  'How the AI industry profits from catastrophe'],
 'agency': [False, True, False],
 'humanComparison': [False, True, False],
 'hyperbole': [False, True, True],
 'historyComparison': [False, False, False],
 'unjustClaims': [False, False, False],
 'deepSounding': [False, False, False],
 'sceptics': [False, False, False],
 'deEmphasize': [False, False, False],
 'performanceNumber': [False, False, False],
 'inscrutable': [False, False, False]}

In [5]:
labels = [label for label in dataset["train"].features.keys() if label not in ["text"]]

num_epochs = {
    "agency": 10,
    "humanComparison": 2,
    "hyperbole": 2,
    "historyComparison": 2,
    "unjustClaims": 5,
    "deepSounding": 2,
    "sceptics": 2,
    "deEmphasize": 7,
    "performanceNumber": 2,
    "inscrutable": 2,
}

labels

['agency',
 'humanComparison',
 'hyperbole',
 'historyComparison',
 'unjustClaims',
 'deepSounding',
 'sceptics',
 'deEmphasize',
 'performanceNumber',
 'inscrutable']

## Preprocess Data, Create Train/Test Split

In [6]:
processed_dataset = {}
for label in labels:
    projected_dataset = (
        dataset["train"]
        .map(remove_columns=[l for l in labels if l != label])
        .rename_column(label, "labels")
        .class_encode_column("labels")
    )
    processed_dataset[label] = projected_dataset.train_test_split(
        test_size=0.2, stratify_by_column="labels"
    )
    # print(f"{label}:\n\t{processed_dataset[label]['test'][0:3]}\n")

processed_dataset

Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-3bce7e24a0318037.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-cf0c2a62c015b92a.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-1cd7d2387c8dd20a.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-c1a6a80eb1005e99.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-cfdee65a4c7aa8fc/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51

{'agency': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'humanComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'hyperbole': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'historyComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 66
     })
 }),
 'unjustClaims': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 263
     })
     t

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [8]:
tokenized_dataset = {
    k: ds.map(
        preprocess_data,
        remove_columns="text",
        batched=True,
    )
    for k, ds in processed_dataset.items()
}

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Verify dataset

In [9]:
example = tokenized_dataset["agency"]["train"][0]
print(example.keys())

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
tokenizer.decode(example["input_ids"])

'[CLS] We built a database to understand the China Initiative. Then the government changed its records. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [11]:
example["labels"]

0

## Load Pre-Trained Model

In [12]:
# use_fast uses fast tokenizers backed by rust. Remove it if it causes errors
# model = AutoModelForSequenceClassification.from_pretrained(
    # "bert-base-cased",
    # num_labels=2,
# )

### Verify data-model interaction

In [13]:
# forward pass
# outputs = model(
# input_ids=tokenized_dataset[labels[0]]["train"]["input_ids"][0],
# labels=tokenized_dataset[labels[0]]["train"][0]["labels"],
# )
# outputs

## Define Metrics

In [14]:
metrics = {
    "accuracy": evaluate.load("accuracy"),
    "presicion": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        name: metric.compute(predictions=predictions, references=labels)
        for name, metric in metrics.items()
    }

In [16]:
class HighPrecisionTrainer(Trainer):
    """A trainer class, which computes loss based on a weighted MSE, where the error for the positive labels is
    weighted more than the error for the negative labels, leading to a higher precision
    """

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        label_mask = torch.FloatTensor(1, 2).cuda().zero_()
        label_mask[0, labels[0]] = 1
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss (suppose one has 3 labels with different weights)
        loss = torch.sum(torch.tensor([[1, 10]]).cuda() * ((logits - label_mask) ** 2))
        return (loss, outputs) if return_outputs else loss

## Train the Model

In [17]:
batch_size = 8  # TODO: increase if we have more data
# metric_name = "f1"

In [18]:
for label in labels:
    print(f"training model for {label}")

    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-cased",
        num_labels=2,
    )

    training_args = TrainingArguments(
        f"aihype_{label}-vs-rest",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs[label],
        weight_decay=0.01,
        report_to="all",
        # load_best_model_at_end=True,
        # metric_for_best_model=metric_name,
        # push_to_hub=True,  # TODO: enable once model seems good
    )

    trainer = HighPrecisionTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset[label]["train"],
        eval_dataset=tokenized_dataset[label]["test"],
        compute_metrics=compute_metrics,
    )

    trainer.train()

training model for agency


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Accuracy,Presicion,Recall,F1
1,No log,242.549164,{'accuracy': 0.7878787878787878},{'precision': 0.5},{'recall': 0.07142857142857142},{'f1': 0.125}
2,No log,305.165833,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
3,No log,296.235077,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
4,No log,249.438324,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
5,No log,274.140289,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
6,No log,309.476990,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
7,No log,321.860321,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
8,No log,299.824524,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
9,No log,293.349670,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
10,No log,286.070953,{'accuracy': 0.7878787878787878},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}


***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
Saving model checkpoint to agency-vs-rest/checkpoint-33
Configuration saved in agency-vs-rest/checkpoint-33/config.json
Model weights saved in agency-vs-rest/checkpoint-33/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to agency-vs-rest/checkpoint-66
Configuration saved in agency-vs-rest/checkpoint-66/config.json
Model weights saved in agency-vs-rest/checkpoint-66/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precis

training model for humanComparison


...

***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to performanceNumber-vs-rest/checkpoint-33
Configuration saved in performanceNumber-vs-rest/checkpoint-33/config.json
Model weights saved in performanceNumber-vs-rest/checkpoint-33/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to performanceNumber-

training model for inscrutable


loading configuration file config.json from cache at /home/xt0r3/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at /home/xt0r3/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/pyt

Epoch,Training Loss,Validation Loss,Accuracy,Presicion,Recall,F1
1,No log,3.747184,{'accuracy': 0.9848484848484849},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}
2,No log,1.876543,{'accuracy': 0.9848484848484849},{'precision': 0.0},{'recall': 0.0},{'f1': 0.0}


***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to inscrutable-vs-rest/checkpoint-33
Configuration saved in inscrutable-vs-rest/checkpoint-33/config.json
Model weights saved in inscrutable-vs-rest/checkpoint-33/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
/home/xt0r3/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to inscrutable-vs-rest/checkpoint-66
Co

## Upload the Model

In [19]:
# agency-vs-rest/checkpoint-263: 0.75 precision, 0.85 recall
#

In [20]:
# trainer.push_to_hub()